# 2. Classification

In [1]:
import os
import time
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import Perceptron
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.activations import relu, selu
from tensorflow.keras.layers import Activation, LeakyReLU, PReLU
from tensorflow.keras.backend import clear_session

## Classification intro

### Supervised learning

- What is supervised learning?
- Training
- Fitting
- Labels
- Error / cost / objective functions

### Classification

- What is classification?
- Different approaches
  - Logistic regression - log loss
  - ANNs, SVMs
  - ...

### No free lunch

- Model selection.
- Parametric / nonparametric models.
- Discriminative / generative modelling.

### Overfitting

- Overfitting
- Underfitting
- Generalisation

### Optimisation

- Hyperparameters

#### Classification with MNIST

- The aim of classification with MNIST
- Get the data
- Recap analysis
  - Size, shape, type
  - Features
  - Missing data, outliers etc.
  - Target feature of dataset (class)

Download the MNIST dataset, getting features X and labels y:

In [2]:
X, y = fetch_openml('mnist_784', return_X_y=True)

We shall look at the number of unique features in the dataset.

In [3]:
n_classes = len(np.unique(y))
print(n_classes)

10


There appear to be 10 unique digit types.

We shall put the data into a NumPy array and analyse the shape of the data:

In [4]:
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Image shape: {X[0].shape}")

X shape: (70000, 784), y shape: (70000,)
Image shape: (784,)


From the shape we can see that we have the full MNIST dataset of 70000 hand written didgets. 

They are yet to be split into a training and testing set and each image has been flattened to a (784,) array rather than the standard (28,28) form.

Now we shall have a look at the image data to see what form it is in.

In [5]:
print(np.min(X[0]), np.max(X[0]))

0.0 255.0


Looking at the minimum and maximum values in appears that pixel are of the standard 0-255 intensity form. 

It is beneficial to some machine learning methods for values to be normalised between 0-1, notably neural networks so we shall do this in the data preparation step.

Finally we shall check the datatypes for features and labels.

In [6]:
print(X.dtype)
print(y.dtype)

float64
object


The features are of float type as expected but it may be beneficial to convert the string type labels to integers.

#### Preparing the data

  - train, val test sets
  - cross validation
  - reshape / retype

As stated earlier normalising the feature values will be beneficial for machine learning later on.

In [7]:
X_full = X / 255.0
print(np.min(X_full), np.max(X_full))

0.0 1.0


We shall also now convert the string type features to integers for use later on.

In [8]:
y_full = y.astype('int64')
print(y.dtype)

object


We will need to create a train, validation and test set before we proceed using sklearns `train_test_split`.

In [9]:
# train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2)

# remove a validation set from the training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [10]:
print(X_full.shape[0])
print(X_train.shape[0])
print(X_val.shape[0])
print(X_test.shape[0])

70000
44800
11200
14000


Now we can train the data and analyse using the validation set before testing on the test set. This will allow us to minimise overfitting by ensuring our model generalises well.

Later on we shall use cross-validation on the entire training set to ensure maximum generalisation, but for now we shall use the the validation set for this.

## 2.1 Artificial Neural Networks (ANNs)

### Neurons

- Neurons structure
- Action potentials ~ activation functions
- Weights, connections etc.

### The perceptron

- Perceptron history
- Input layer
- Output layer
- Activation function
- Weights
- Bias
- Try and apply it to MNIST (lots of neurons?)

In [59]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

Perceptron()

In [60]:
perceptron.score(X_val, y_val)

0.8815178571428571

### Perceptron regularisation

This is a reasonably good result for such a simple model, but this is due to the simplicity of the data set.

One way we can try and improve the results is through regularisation.

** **EXPLAIN REGULARISATION** **

As the model training doesn't take too long we shall try a grid search with `None`, `l1`, `l2` and `ElastiNet` regularisation.

We shall do this with 5-fold cross validation, evaluating the accuracy on the validation set.

** **EXPLAIN CROSS VALIDATION** **

In [43]:
reg_perceptron = Perceptron(alpha=0.0001, tol=0.001,
                            max_iter=1000, early_stopping=True,
                            validation_fraction=0.1, n_iter_no_change=5,
                            random_state=42, verbose=0, n_jobs=-1)

perceptron_params = [{'penalty': ['None', 'l1', 'l2', 'elasticnet']}]

grid_search = GridSearchCV(reg_perceptron, perceptron_params,
                           scoring='accuracy', cv=5,
                           return_train_score=True,
                           verbose=3)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] penalty=None ....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... penalty=None, score=(train=0.866, test=0.848), total=   2.2s
[CV] penalty=None ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] .... penalty=None, score=(train=0.883, test=0.870), total=   2.1s
[CV] penalty=None ....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.4s remaining:    0.0s


[CV] .... penalty=None, score=(train=0.876, test=0.870), total=   2.1s
[CV] penalty=None ....................................................
[CV] .... penalty=None, score=(train=0.885, test=0.870), total=   2.1s
[CV] penalty=None ....................................................
[CV] .... penalty=None, score=(train=0.853, test=0.841), total=   2.2s
[CV] penalty=l1 ......................................................
[CV] ...... penalty=l1, score=(train=0.877, test=0.860), total=   2.2s
[CV] penalty=l1 ......................................................
[CV] ...... penalty=l1, score=(train=0.885, test=0.877), total=   2.3s
[CV] penalty=l1 ......................................................
[CV] ...... penalty=l1, score=(train=0.872, test=0.865), total=   2.3s
[CV] penalty=l1 ......................................................
[CV] ...... penalty=l1, score=(train=0.885, test=0.875), total=   2.3s
[CV] penalty=l1 ......................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   45.6s finished


GridSearchCV(cv=5,
             estimator=Perceptron(early_stopping=True, n_jobs=-1,
                                  random_state=42),
             param_grid=[{'penalty': ['None', 'l1', 'l2', 'elasticnet']}],
             return_train_score=True, scoring='accuracy', verbose=3)

We can now check the optimal parameters of the grid search.

In [44]:
grid_search.best_params_

{'penalty': 'l1'}

It appears l1 regularisation has produced the best results for the MNIST dataset.

### Multi-layer perceptron

- Multiple layers of neurons
- Able to learn more complex patterns

### Training MLPs

- Forward pass
- Backward pass
- Epochs, convergence

### Forward pass

- Calculation of outputs from input
- Calculates loss

### Backpropogation

- Gradient descent
- Learning rate
- Minimise loss function
- Convex optimisation problem
- Chain rule

### Stochastic gradient descent

- What is SGD
- Diagram

### Cross entropy

- What is cross entropy
- Sparse categorical cross entropy

### Classifying MNIST

- Feed forward network
- Dense layers
- Epochs, convergence
- Create a simple MLP for MNIST

In [61]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=[784]))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd', metrics=["accuracy"])

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_10 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                510       
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [63]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Train on 44800 samples, validate on 11200 samples
Epoch 1/10
44800/44800 [==============================] - 9s 190us/sample - loss: 0.7054 - accuracy: 0.8126 - val_loss: 0.3659 - val_accuracy: 0.8951
Epoch 2/10
44800/44800 [==============================] - 7s 167us/sample - loss: 0.3244 - accuracy: 0.9065 - val_loss: 0.2966 - val_accuracy: 0.9142
Epoch 3/10
44800/44800 [==============================] - 8s 183us/sample - loss: 0.2734 - accuracy: 0.9206 - val_loss: 0.2621 - val_accuracy: 0.9229
Epoch 4/10
44800/44800 [==============================] - 8s 179us/sample - loss: 0.2413 - accuracy: 0.9305 - val_loss: 0.2313 - val_accuracy: 0.9306
Epoch 5/10
44800/44800 [==============================] - 9s 193us/sample - loss: 0.2161 - accuracy: 0.9379 - val_loss: 0.2104 - val_accuracy: 0.9385
Epoch 6/10
44800/44800 [==============================] - 8s 168us/sample - loss: 0.1953 - accuracy: 0.9432 - val_loss: 0.1974 - val_accuracy: 0.9411
Epoch 7/10
44800/44800 [==========================

### Analysing model training

We shall use Tensorboard to visualise our model results as well as analyse training.

We shall configure it to create a new subdirectory for each model instance.

In [11]:
def get_tb_dir():
    curr_dir = os.path.join(os.curdir, "tensorboard_logs")
    tb_dir = time.strftime("model_%Y_%m_%d-%H-%M-%S")
    return os.path.join(curr_dir, tb_dir)

Now we can create a callback during model training for Tensorboard

In [12]:
tensorboard = TensorBoard(get_tb_dir())

Fitting the model with the callback will then write the logs to it's own directory in `tensorboard_logs`.

In [74]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=[784]))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd', metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard])

Train on 44800 samples, validate on 11200 samples
Epoch 1/10
44800/44800 [==============================] - 8s 179us/sample - loss: 0.7610 - accuracy: 0.8044 - val_loss: 0.3651 - val_accuracy: 0.8956
Epoch 2/10
44800/44800 [==============================] - 8s 185us/sample - loss: 0.3232 - accuracy: 0.9075 - val_loss: 0.2893 - val_accuracy: 0.9163
Epoch 3/10
44800/44800 [==============================] - 8s 180us/sample - loss: 0.2702 - accuracy: 0.9229 - val_loss: 0.2627 - val_accuracy: 0.9226
Epoch 4/10
44800/44800 [==============================] - 8s 170us/sample - loss: 0.2387 - accuracy: 0.9313 - val_loss: 0.2281 - val_accuracy: 0.9337
Epoch 5/10
44800/44800 [==============================] - 8s 174us/sample - loss: 0.2151 - accuracy: 0.9384 - val_loss: 0.2086 - val_accuracy: 0.9376
Epoch 6/10
44800/44800 [==============================] - 8s 183us/sample - loss: 0.1963 - accuracy: 0.9435 - val_loss: 0.1923 - val_accuracy: 0.9430
Epoch 7/10
44800/44800 [==========================

Now we can look at the Tensorboard logs

In [91]:
# !kill 132185
# %reload_ext tensorboard
# %tensorboard --logdir=./tensorboard_logs --port=6006

### Deeper neural networks

- Create a large overfitting network
- Early stopping, check pointing

In [16]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=[784]))
for i in range(30):
    model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd', metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_65 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_66 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_67 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_68 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_69 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_70 (Dense)             (None, 100)              

In [13]:
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard, early_stopping])

As we can see this large model with almost 400,000 parameters was a lot harder to train and generalised far worse on the validation set.

### Vanishing / exploding gradient problem

- What is the vanishing gradient problem

I shall create a Keras wrapper in order to use Scikit's grid search.

With this wrapper we can search for params: 
- `kernel_initializer`: weigh and bias initialisation method
- `optimizer`: optimisation method used in back progogation
- `activation`: activation function used in forward pass
- `lr`: learning rate of the optimiser
- `momentum`: momentum of the optimiser

### Initlialisation

- GloRoot
- He
- LeCeun

In [29]:
# Function to create model, required for KerasClassifier
def create_init_model(init):
    # Sequential model
    model = Sequential()
    # Input layer with activation layer
    model.add(Dense(100, kernel_input_shape=[784]))
    model.add(get_activation(activation))
    # Hidden layers with activation function
    for i in range(21):
        model.add(Dense(50))
        model.add(get_activation(activation))
    # Output layer
    model.add(Dense(10, activation='sigmoid'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd',
                  metrics=['accuracy'])
    return model

In [ ]:
init_model = KerasClassifier(build_fn=create_init_model, epochs=10)
param_grid = {"init": ['lecun_uniform', 'glorot_normal', 'glorot_uniform',
                       'he_normal', 'he_uniform']}

### Non-saturating activation functions

- Leaky RELU
- SELU

In [27]:
# Return activation function as layer
def get_activation(activation):
    if (activation == 'relu'):
        return Activation(relu)
    elif (activation == 'selu'):
        return Activation(selu)
    elif (activation == 'leakyrelu'):
        return LeakyReLU()
    elif (activation == 'prelu'):
        return PReLU()
    return False

In [28]:
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
    # Sequential model
    model = Sequential()
    # Input layer with activation layer
    model.add(Dense(100, input_shape=[784]))
    model.add(get_activation(activation))
    # Hidden layers with activation function
    for i in range(21):
        model.add(Dense(50))
        model.add(get_activation(activation))
    # Output layer
    model.add(Dense(10, activation='sigmoid'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd',
                  metrics=['accuracy'])
    return model

In [23]:
np.random.seed(42)
act_model = KerasClassifier(build_fn=create_model, epochs=10)
param_grid = {"activation": ['relu', 'selu', 'leakyrelu', 'prelu']}

# grid search
act_grid = RandomizedSearchCV(act_model, param_grid, n_iter=4, cv=2, verbose=0)
act_grid_results = act_grid.fit(X_train, y_train,
                                validation_data=[X_val, y_val],
                                callbacks=[tensorboard, early_stopping],
                                verbose=0)

In [24]:
act_grid_results.best_params_

{'activation': 'prelu'}

### Batching

- Batch normalisation
- Mini-batch
- Implement batching

In [32]:
model = Sequential()
model.add(BatchNormalization(input_shape=[784]))
model.add(Dense(100, activation="relu"))
for i in range(20):
    model.add(BatchNormalization())
    model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd', metrics=["accuracy"])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_21 (Batc (None, 784)               3136      
_________________________________________________________________
dense_50 (Dense)             (None, 100)               78500     
_________________________________________________________________
batch_normalization_22 (Batc (None, 100)               400       
_________________________________________________________________
dense_51 (Dense)             (None, 50)                5050      
_________________________________________________________________
batch_normalization_23 (Batc (None, 50)                200       
_________________________________________________________________
dense_52 (Dense)             (None, 50)                2550      
_________________________________________________________________
batch_normalization_24 (Batc (None, 50)              

In [33]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=[X_val, y_val],
                    callbacks=[tensorboard, early_stopping])

Train on 44800 samples, validate on 11200 samples
Epoch 1/20
44800/44800 [==============================] - 46s 1ms/sample - loss: 1.4566 - accuracy: 0.5210 - val_loss: 0.8362 - val_accuracy: 0.7379
Epoch 2/20
44800/44800 [==============================] - 43s 956us/sample - loss: 0.8345 - accuracy: 0.7538 - val_loss: 0.4975 - val_accuracy: 0.8545
Epoch 3/20
44800/44800 [==============================] - 39s 861us/sample - loss: 0.6466 - accuracy: 0.8164 - val_loss: 0.4047 - val_accuracy: 0.8841
Epoch 4/20
44800/44800 [==============================] - 40s 886us/sample - loss: 0.5597 - accuracy: 0.8455 - val_loss: 0.3630 - val_accuracy: 0.9006
Epoch 5/20
44800/44800 [==============================] - 39s 865us/sample - loss: 0.4906 - accuracy: 0.8644 - val_loss: 0.3184 - val_accuracy: 0.9176
Epoch 6/20
44800/44800 [==============================] - 41s 913us/sample - loss: 0.4326 - accuracy: 0.8840 - val_loss: 0.2627 - val_accuracy: 0.9315
Epoch 7/20
44800/44800 [======================

### Optimisers

- Momentum
    - For SGD
- Better optimisers
    - Adam
    - Compare some others?

#### Momentum

In [34]:
model = Sequential()
model.add(BatchNormalization(input_shape=[784]))
model.add(Dense(100, activation="relu"))
for i in range(20):
    model.add(BatchNormalization())
    model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=SGD(momentum=0.9), metrics=["accuracy"])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_42 (Batc (None, 784)               3136      
_________________________________________________________________
dense_72 (Dense)             (None, 100)               78500     
_________________________________________________________________
batch_normalization_43 (Batc (None, 100)               400       
_________________________________________________________________
dense_73 (Dense)             (None, 50)                5050      
_________________________________________________________________
batch_normalization_44 (Batc (None, 50)                200       
_________________________________________________________________
dense_74 (Dense)             (None, 50)                2550      
_________________________________________________________________
batch_normalization_45 (Batc (None, 50)              

In [35]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=[X_val, y_val],
                    callbacks=[tensorboard, early_stopping])

Train on 44800 samples, validate on 11200 samples
Epoch 1/20
44800/44800 [==============================] - 36s 813us/sample - loss: 1.1458 - accuracy: 0.6124 - val_loss: 0.5969 - val_accuracy: 0.8218
Epoch 2/20
44800/44800 [==============================] - 31s 683us/sample - loss: 0.6835 - accuracy: 0.7956 - val_loss: 0.3877 - val_accuracy: 0.8945
Epoch 3/20
44800/44800 [==============================] - 31s 691us/sample - loss: 0.5391 - accuracy: 0.8470 - val_loss: 0.3834 - val_accuracy: 0.8759
Epoch 4/20
44800/44800 [==============================] - 31s 691us/sample - loss: 0.4572 - accuracy: 0.8723 - val_loss: 0.2871 - val_accuracy: 0.9265
Epoch 5/20
44800/44800 [==============================] - 31s 702us/sample - loss: 0.3907 - accuracy: 0.8923 - val_loss: 0.2648 - val_accuracy: 0.9343
Epoch 6/20
44800/44800 [==============================] - 31s 686us/sample - loss: 0.3557 - accuracy: 0.9040 - val_loss: 0.2553 - val_accuracy: 0.9379
Epoch 7/20
44800/44800 [====================

#### Adaptive moment esitmation (Adam)

In [14]:
adam_model = Sequential()
adam_model.add(Dense(100, activation="relu"))
adam_model.add(BatchNormalization(input_shape=[784]))
for i in range(20):
    adam_model.add(Dense(50, activation="relu"))
    adam_model.add(BatchNormalization())
adam_model.add(Dense(10, activation="softmax"))

adam_model.compile(loss="sparse_categorical_crossentropy",
                   optimizer=Adam(), metrics=["accuracy"])

In [ ]:
clear_session()

history = adam_model.fit(X_train, y_train, epochs=20,
                         validation_data=[X_val, y_val],
                         callbacks=[tensorboard, early_stopping])

Train on 44800 samples, validate on 11200 samples
Epoch 1/20
44800/44800 [==============================] - 45s 994us/sample - loss: 1.8571 - accuracy: 0.3513 - val_loss: 1.0240 - val_accuracy: 0.6459
Epoch 2/20
44800/44800 [==============================] - 40s 892us/sample - loss: 0.9662 - accuracy: 0.6927 - val_loss: 0.4787 - val_accuracy: 0.8678
Epoch 3/20
44800/44800 [==============================] - 38s 856us/sample - loss: 0.6507 - accuracy: 0.8150 - val_loss: 0.3270 - val_accuracy: 0.9131
Epoch 4/20
44800/44800 [==============================] - 39s 881us/sample - loss: 0.5140 - accuracy: 0.8610 - val_loss: 0.2956 - val_accuracy: 0.9269
Epoch 5/20
44800/44800 [==============================] - 38s 859us/sample - loss: 0.4149 - accuracy: 0.8914 - val_loss: 0.2656 - val_accuracy: 0.9362
Epoch 6/20
44800/44800 [==============================] - 38s 858us/sample - loss: 0.3666 - accuracy: 0.9047 - val_loss: 0.2423 - val_accuracy: 0.9437
Epoch 7/20
44800/44800 [====================

### Minibatch Adam optimisation

In [15]:
clear_session()

history = adam_model.fit(X_train, y_train, epochs=20, batch_size=16,
                         validation_data=[X_val, y_val],
                         callbacks=[tensorboard, early_stopping])

Epoch 1/20
2799/2800 [============================>.] - ETA: 0s - loss: 1.9125 - accuracy: 0.3221

ValueError: in user code:

    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:909 test_step  **
        y_pred = self(x, training=False)
    /home/kai/anaconda3/envs/tf_gpu/lib/python3.8/sit

#### Learning rate

In [37]:
def create_lr_model(lr):
    model = Sequential()
    model.add(BatchNormalization(input_shape=[784]))
    model.add(Dense(100, activation="relu"))
    for i in range(20):
        model.add(BatchNormalization())
        model.add(Dense(50, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=SGD(lr=lr, momentum=0.9), metrics=["accuracy"])

In [ ]:
np.random.seed(42)
lr_model = KerasClassifier(build_fn=create_model, epochs=10)
param_grid = {"lr": []}

# grid search
lr_grid = RandomizedSearchCV(act_model, param_grid, n_iter=4, cv=2, verbose=0)
lr_grid_results = act_grid.fit(X_train, y_train,
                               validation_data=[X_val, y_val],
                               callbacks=[tensorboard, early_stopping],
                               verbose=0)

### Learning rate

- Analyse adjusting learning rate
- Learning rate schedling

### Overfitting in ANNs

- Why large networks can overfit
- l1, l2 loss and regularisation
- Implement regularisation
- Dropout, monte carlo dropout
- Create a network with dropout

In [22]:
drop_model = Sequential()

drop_model.add(BatchNormalization(input_shape=[784]))
drop_model.add(Dense(100, activation="relu"))

for i in range(20):
    drop_model.add(Dense(50, activation="relu"))
    drop_model.add(BatchNormalization())

drop_model.add(Dense(50, activation="relu"))
drop_model.add(Dropout(rate=0.5))

drop_model.add(Dense(10, activation="softmax"))

drop_model.compile(loss="sparse_categorical_crossentropy",
                   optimizer='sgd', metrics=["accuracy"])

In [23]:
clear_session()

history = drop_model.fit(X_train, y_train, epochs=20,
                         validation_data=[X_val, y_val],
                         callbacks=[tensorboard, early_stopping])

Train on 44800 samples, validate on 11200 samples
Epoch 1/20
44800/44800 [==============================] - 40s 892us/sample - loss: 1.7954 - accuracy: 0.3976 - val_loss: 1.0093 - val_accuracy: 0.6487
Epoch 2/20
44800/44800 [==============================] - 33s 746us/sample - loss: 1.1686 - accuracy: 0.6336 - val_loss: 0.7085 - val_accuracy: 0.7703
Epoch 3/20
44800/44800 [==============================] - 33s 726us/sample - loss: 0.9309 - accuracy: 0.7276 - val_loss: 0.5051 - val_accuracy: 0.8550
Epoch 4/20
44800/44800 [==============================] - 33s 733us/sample - loss: 0.7723 - accuracy: 0.7997 - val_loss: 0.3993 - val_accuracy: 0.8919
Epoch 5/20
44800/44800 [==============================] - 33s 731us/sample - loss: 0.6710 - accuracy: 0.8310 - val_loss: 0.3502 - val_accuracy: 0.9045
Epoch 6/20
44800/44800 [==============================] - 32s 721us/sample - loss: 0.5939 - accuracy: 0.8524 - val_loss: 0.3058 - val_accuracy: 0.9213
Epoch 7/20
44800/44800 [====================

### Convolutional neural networks (CNN)

- Problems with neural networks and images
- Convolutions, filters
- Convolutional layers, feature maps
- Simple CNN for MNIST

### Training better CNNs

- Stride, step
- Pooling, max pooling
- Better CNN

### Transfer learning

- What is transfer learning
- Use transfer learning with MNIST

### Limitations of ANNs

- Problems with ANNs

## 2.2. Support Vector Machines (SVMs)

Train an SVM (with a chosen Kernel) and perform the same analyses as for ANNs. Interpret  and  discuss  your  results. Does the model overfit? How do they compare with ANNs? And why? How does the type of kernel (e.g.linear, RBF, etc.) impact on performance?

### Support vector machines

- Hyperplane
- Support vectors
- Minimising lagrange multiplier (MIT)

### Linear SVM

- Implementation of SVM on MNIST
- Plot decision boundaries from lab 2

### Probabilistic SVM

- What is a probabilistic SVM?
- SVM on MNIST

### The dual problem

What is the dual problem

### Kernels

- Different representations
- Kernels
- Mercer's theorem
- Gram matrix

### The kernel trick

- What is the kernel trick

### Kernel SVMs

- How kernel SVMs work

### Polynomial KSVM

- What is the polynomial kernel
- Implementation

### Radial basis function (RBF) KSVM

- What is the RBF kernel
- Implementation

### Sigmoid KSVM

- What is the sigmoid kernel
- Implementation

### Optimising SVMs

- Hinge loss
- Grid search hyperparameters

### Limitations of SVMs

- Problems with SVMs

## Classification conclusion

#### Comparison of ANNs and SVMs

- Comparison of neural networks and support vector machines.
- Area under ROC curve
- Precision, recall, accuracy...
- TP, TN, FP, FN and rates for each
- Confusion matrix
- Metric
- Validation
- Cross-validation
- Prediction
- Inference
- Interpretability
- Sensitivity vs specificity